In [1]:
from ragas.testset.prompts import (
    context_scoring_prompt,
    evolution_elimination_prompt,
    filter_question_prompt,
)
from langchain_community.document_loaders import DirectoryLoader
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context

### Interference server setting

In [ ]:
##python -m vllm.entrypoints.openai.api_server --model explodinggradients/Ragas-critic-llm-Qwen1.5-GPTQ

# inference_server_url = "http://localhost:8000/v1"
# MODEL = "explodinggradients/Ragas-critic-llm-Qwen1.5-GPTQ"
# chat = ChatOpenAI(
#     model=MODEL,
#     openai_api_key="token-abc123",
#     openai_api_base=inference_server_url,
#     max_tokens=2048,
#     temperature=0,
# )

In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
# from langchain_core.language_models import BaseLanguageModel
# from langchain_core.embeddings import Embeddings


# langchain_llm =  # any langchain LLM instance
# langchain_embeddings = # any langchain Embeddings instance


# langchain_llm = ChatOllama(model="llama3:latest")
# langchain_llm = ChatOllama(base_url="http://localhost:11434", model="phi3", streaming=True)
langchain_llm = ChatOllama(base_url="http://localhost:11434", model="mistral")
langchain_embeddings = OllamaEmbeddings(base_url="http://localhost:11434", model="mistral")


# langchain_llm.invoke("What is stock?")
# results = evaluate(metrics=[], llm=langchain_llm, embeddings=langchain_embeddings)


In [14]:
from datasets import Dataset 
from ragas.metrics import context_precision
from ragas import evaluate

data_samples = {
    'question': ['When was the first super bowl?', 'Who won the most super bowls?'],
    'answer': ['The first superbowl was held on Jan 15, 1967', 'The most super bowls have been won by The New England Patriots'],
    'contexts' : [['The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles,'], 
    ['The Green Bay Packers...Green Bay, Wisconsin.','The Packers compete...Football Conference']],
    'ground_truth': ['The first superbowl was held on January 15, 1967', 'The New England Patriots have won the Super Bowl a record six times']
}
dataset = Dataset.from_dict(data_samples)
# score = evaluate(dataset,metrics=[context_precision])
score = evaluate(dataset, llm=langchain_llm, embeddings=langchain_embeddings, metrics=[context_precision])
df_score = score.to_pandas()
df_score

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

,question,answer,contexts,ground_truth,context_precision
0,When was the first super bowl?,"The first superbowl was held on Jan 15, 1967",[The First AFL–NFL World Championship Game was...,"The first superbowl was held on January 15, 1967",1.0
1,Who won the most super bowls?,The most super bowls have been won by The New ...,"[The Green Bay Packers...Green Bay, Wisconsin....",The New England Patriots have won the Super Bo...,0.5


In [15]:
df_score

,question,answer,contexts,ground_truth,context_precision
0,When was the first super bowl?,"The first superbowl was held on Jan 15, 1967",[The First AFL–NFL World Championship Game was...,"The first superbowl was held on January 15, 1967",1.0
1,Who won the most super bowls?,The most super bowls have been won by The New ...,"[The Green Bay Packers...Green Bay, Wisconsin....",The New England Patriots have won the Super Bo...,0.5


### Generation of test data

In [ ]:
# remove demonstrations from examples
for prompt in [
    context_scoring_prompt,
    evolution_elimination_prompt,
    filter_question_prompt,
]:
    prompt.examples = []

In [ ]:
from ragas.testset.filters import QuestionFilter, EvolutionFilter, NodeFilter


from ragas.llms import LangchainLLMWrapper

langchain_llm = LangchainLLMWrapper(chat)

qa_filter = QuestionFilter(langchain_llm, filter_question_prompt)
node_filter = NodeFilter(langchain_llm, context_scoring_prompt=context_scoring_prompt)
evolution_filter = EvolutionFilter(langchain_llm, evolution_elimination_prompt)

In [ ]:
distributions = {simple: 0.5, reasoning: 0.25, multi_context: 0.25}

In [ ]:
# customise the filters
from ragas.testset.evolutions import ComplexEvolution

for evolution in distributions:
    if evolution.question_filter is None:
        evolution.question_filter = qa_filter
    if evolution.node_filter is None:
        evolution.node_filter = node_filter

    if isinstance(evolution, ComplexEvolution):
        if evolution.evolution_filter is None:
            evolution.evolution_filter = evolution_filter

In [ ]:
from renumics import spotlight

spotlight.show(df, dtype={"image": spotlight.Image, "embedding": spotlight.Embedding})

In [16]:
import pandas as pd
from renumics import spotlight

# df = df_score.copy()
df = pd.read_csv("https://renumics.com/data/mnist/mnist-tiny.csv")
spotlight.show(df, dtype={"image": spotlight.Image})

2024-07-31 02:46:34.080 | WARNING  | renumics.spotlight.analysis.analyzers.cleanlab:<module>:20 - Cleanlab analyzer requires `cleanlab` to be installed.
2024-07-31 02:46:34.081 | WARNING  | renumics.spotlight.analysis.analyzers.cleanvision:<module>:24 - Cleanvision analyzer requires `cleanvision` to be installed.
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [17]:
df

,question,answer,contexts,ground_truth,context_precision
0,When was the first super bowl?,"The first superbowl was held on Jan 15, 1967",[The First AFL–NFL World Championship Game was...,"The first superbowl was held on January 15, 1967",1.0
1,Who won the most super bowls?,The most super bowls have been won by The New ...,"[The Green Bay Packers...Green Bay, Wisconsin....",The New England Patriots have won the Super Bo...,0.5
